In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# --------------------------
# 1. Simulated Swap Rates
# --------------------------
np.random.seed(42)
dates = pd.date_range(start="2007-01-01", periods=4000, freq="B")
tenors = ['ON', '1W', '1M', '3M', '6M', '1Y', '2Y', '3Y', '4Y', '5Y',
          '7Y', '10Y', '15Y', '20Y', '25Y', '30Y', '40Y', '50Y', '60Y', '70Y']

data = pd.DataFrame(np.random.normal(0.03, 0.005, size=(len(dates), len(tenors))), index=dates, columns=tenors)

# --------------------------
# 2. PCA Tilt Vector (PC2)
# --------------------------
returns = data.diff().dropna()
returns_scaled = StandardScaler().fit_transform(returns)

pca = PCA(n_components=3)
pca.fit(returns_scaled)
pca_tilt_vector = pca.components_[1]
pca_tilt_vector /= np.linalg.norm(pca_tilt_vector)

# --------------------------
# 3. Stylized Tilt Vector (linear slope around 10Y)
# --------------------------
def tenor_to_years(tenor):
    if tenor.endswith('Y'): return int(tenor[:-1])
    if tenor.endswith('M'): return int(tenor[:-1]) / 12
    if tenor.endswith('W'): return int(tenor[:-1]) / 52
    if tenor == 'ON': return 1 / 252
    return np.nan

tenor_years = np.array([tenor_to_years(t) for t in tenors])
pivot = 10
stylized_tilt = tenor_years - pivot
stylized_tilt /= np.linalg.norm(stylized_tilt)

# --------------------------
# 4. Autoencoder Tilt Vector
# --------------------------
input_dim = len(tenors)
encoding_dim = 3

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='linear')(input_layer)
decoded = Dense(input_dim, activation='linear')(encoded)
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

autoencoder.fit(returns_scaled, returns_scaled, epochs=50, batch_size=64, verbose=0)

encoder = Model(input_layer, encoded)
weights = encoder.layers[1].get_weights()[0]  # shape = (input_dim, encoding_dim)
autoencoder_tilt_vector = weights[:, 1]
autoencoder_tilt_vector /= np.linalg.norm(autoencoder_tilt_vector)

# --------------------------
# 5. Combine Results
# --------------------------
tilt_vectors = pd.DataFrame({
    "Tenor": tenors,
    "Years": tenor_years,
    "Stylized Tilt": stylized_tilt,
    "PCA Tilt": pca_tilt_vector,
    "Autoencoder Tilt": autoencoder_tilt_vector
})

tilt_vectors.set_index("Tenor", inplace=True)
print(tilt_vectors)

# Optional plot
tilt_vectors[["Stylized Tilt", "PCA Tilt", "Autoencoder Tilt"]].plot(title="Comparison of Tilt Vectors", figsize=(12, 6))
plt.ylabel("Relative Weight")
plt.grid(True)
plt.show()
